In [1]:
import pandas as pd
import numpy as np

# Load the data from csv
data = pd.read_csv('Master Document of H-2A Orders - Hit or Not - Previous Semester H-2A Orders Successes.csv')

# print the first 5 rows of the data
print(data.head())


          Case Number                                       Link to H-2A  \
0  H-300-21280-631097  https://seasonaljobs.dol.gov/api/job-order/H-3...   
1  H-300-21202-474481  https://seasonaljobs.dol.gov/api/job-order/H-3...   
2  H-300-21195-461870  https://seasonaljobs.dol.gov/api/job-order/H-3...   
3  H-300-21183-441615  https://seasonaljobs.dol.gov/api/job-order/H-3...   
4  H-300-20255-817429  https://seasonaljobs.dol.gov/api/job-order/H-3...   

  Hit or No Hit Market Farm Narratives  \
0        No Hit    NaN  NaN        NaN   
1        No Hit    NaN  NaN        NaN   
2        No Hit    NaN  NaN        NaN   
3        No Hit    NaN  NaN        NaN   
4        No Hit    NaN  NaN        NaN   

  Facebook. Google Reviews, Website, LinkedIn, Google Maps, Instagram, etc.???  \
0                                                NaN                             
1                                                NaN                             
2                                           

In [2]:
# make a df of just rows with "hit"
hit = data[data['Hit or No Hit'] == 'Hit']

# print the first 5 rows of the hit data
print(hit.head())

           Case Number                                       Link to H-2A  \
13  H-300-21029-042637  https://seasonaljobs.dol.gov/api/job-order/H-3...   
35  H-300-20013-248635  https://seasonaljobs.dol.gov/api/job-order/H-3...   
38  H-300-20002-228933  https://seasonaljobs.dol.gov/api/job-order/H-3...   
44  H-300-19044-646927  https://seasonaljobs.dol.gov/jobs/H-300-19044-...   
48  H-300-19114-230380  https://seasonaljobs.dol.gov/jobs/H-300-19114-...   

   Hit or No Hit                                             Market  \
13           Hit                        Kroger, Walmart, Instacart    
35           Hit  Naturipe Farms, 99 Cents Only Stores, Ahold, A...   
38           Hit  Store Locator – sealtheseasons, Peoples-Food A...   
44           Hit                             https://wishfarms.com/   
48           Hit                                      Harris Teeter   

                             Farm  \
13                  Haigler Farms   
35         Down South Berries LLC   

In [3]:
# save the hit data to a new csv
hit.to_csv('hit.csv', index=False)

In [31]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Load the CSV file
file_path = "hit.csv"
df = pd.read_csv(file_path)

# Column containing URLs
column_name = "Link to H-2A"
urls = df[column_name].dropna().unique()

# Create folder for PDFs
output_folder = "H2A_PDFs"
os.makedirs(output_folder, exist_ok=True)

# Set up Chrome WebDriver (Headless Mode)
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--log-level=3")  

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Track successes and failures
successful_downloads = []
failed_urls = []

# Function to check if a URL is a direct PDF
def is_pdf(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=10)
        content_type = response.headers.get("Content-Type", "")

        if not content_type or "application/pdf" not in content_type:
            response = requests.get(url, stream=True, timeout=10)
            content_type = response.headers.get("Content-Type", "")

        return "application/pdf" in content_type
    except requests.exceptions.RequestException:
        return False 

# Function to extract PDF link from a webpage
def extract_pdf_from_page(url):
    try:
        driver.get(url)
        time.sleep(2)  

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Try extracting direct PDF links
        for link in soup.find_all("a", href=True):
            text = link.text.strip().lower()
            if "view job order" in text or link["href"].endswith(".pdf"):
                return urljoin(url, link["href"])

        # Look for PDFs in <iframe>
        iframe = soup.find("iframe", src=True)
        if iframe and iframe["src"].endswith(".pdf"):
            return urljoin(url, iframe["src"])

    except Exception as e:
        print(f"❌ Error processing webpage {url}: {e}")
    return None

# Function to download a PDF and track success
def download_pdf(pdf_url, source_url):
    try:
        response = requests.get(pdf_url, stream=True, timeout=10)
        content_type = response.headers.get("Content-Type", "")

        if "application/pdf" not in content_type:
            print(f"⚠️ Not a PDF: {pdf_url} (Content-Type: {content_type})")
            failed_urls.append(source_url)
            return

        filename = os.path.basename(pdf_url)
        if not filename.endswith(".pdf"):
            filename += ".pdf"  

        file_path = os.path.join(output_folder, filename)

        # Write PDF file
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)

        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
            print(f"✅ Downloaded: {file_path}")
            successful_downloads.append(file_path)
        else:
            print(f"❌ Failed to save PDF properly: {file_path}")
            failed_urls.append(source_url)

    except requests.exceptions.RequestException as e:
        print(f"❌ Error downloading PDF from {source_url}: {e}")
        failed_urls.append(source_url)

# Function to process each URL
def process_url(url):
    global driver
    try:
        if is_pdf(url):  
            download_pdf(url, url)
        else:
            pdf_url = extract_pdf_from_page(url)
            if pdf_url:
                download_pdf(pdf_url, url)
            else:
                print(f"⚠️ No 'View Job Order' PDF found on webpage: {url}")
                failed_urls.append(url)

    except Exception as e:
        print(f"⚠️ WebDriver error, restarting: {e}")
        driver.quit()
        driver = webdriver.Chrome(service=service, options=chrome_options)

# Use multi-threading for faster downloads
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_url, urls)

# Close the Selenium browser
driver.quit()

# Print final download summary
print("\n📊 **Download Summary**")
print(f"✅ Successfully downloaded PDFs: {len(successful_downloads)}")
print(f"❌ Failed URLs: {len(failed_urls)}\n")

if successful_downloads:
    print("✅ **Successfully downloaded PDFs:**")
    for file in successful_downloads:
        print(file)

if failed_urls:
    print("\n⚠️ **Failed to download PDFs from these URLs:**")
    for url in failed_urls:
        print(url)

# Debugging output
print("\n🔍 Debugging successful_downloads list:")
print(successful_downloads)

✅ Downloaded: H2A_PDFs/H-300-20013-248635.pdf
✅ Downloaded: H2A_PDFs/H-300-21029-042637.pdf
✅ Downloaded: H2A_PDFs/H-300-20002-228933.pdf
⚠️ Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19114-230380 (Content-Type: text/html; charset=utf-8)
⚠️ Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19044-646927 (Content-Type: text/html; charset=utf-8)
⚠️ Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19068-891355 (Content-Type: text/html; charset=utf-8)
✅ Downloaded: H2A_PDFs/H-300-20357-975271.pdf
✅ Downloaded: H2A_PDFs/H-300-21006-998351.pdf
✅ Downloaded: H2A_PDFs/H-300-22014-831909.pdf
✅ Downloaded: H2A_PDFs/H-300-20009-241352.pdf
⚠️ Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19106-808682 (Content-Type: text/html; charset=utf-8)
⚠️ Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19051-356435 (Content-Type: text/html; charset=utf-8)
⚠️ Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-18254-022527 (Content-Type: text/html; charset=utf-8)
✅ Downloaded: H2A_PDFs/104779_